大模型LayerNorm、BatchNorm、RMSNorm区别

In [ ]:
import torch.nn as nn
import torch

class LayerNorm(nn.Module):
    def __init__(self, dim, epsilon=1e-5):
        super().__init__()
        self.epsilon = epsilon
        self.beta = nn.Parameter(torch.zeros([dim]))
        self.gamma = nn.Parameter(torch.ones([dim]))


    def forward(self, hidden_states):
        mean = hidden_states.mean(dim=-1, keepdim=True)
        var = hidden_states.var(dim=-1, keepdim=True, unbiased=False)
        norm_hidden_states = self.gamma * (hidden_states - mean) / torch.sqrt(var + self.epsilon) + self.beta
        return norm_hidden_states


In [ ]:
import torch.nn as nn
import torch

class RMSNorm(nn.Module):
    def __init__(self, dim, epsilon=1e-5):
        super().__init__()
        self.epsilon = epsilon
        self.gamma = nn.Parameter(torch.ones([dim]))

    def forward(self, hidden_states):
        var = hidden_states.pow(2).mean(dim=-1, keepdim=True)
        norm_hidden_states = self.gamma * hidden_states / torch.sqrt(var + self.epsilon)
        return norm_hidden_states


In [ ]:
import torch
import torch.nn as nn

class BatchNorm(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.eps = eps
        self.momentum = momentum

        # 可学习的缩放和平移参数
        self.gamma = nn.Parameter(torch.ones(num_features))  # scale
        self.beta = nn.Parameter(torch.zeros(num_features))  # shift

        # 运行时的全局均值与方差（用于推理）
        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))

    def forward(self, x):
        if self.training:
            # 按 batch 计算均值与方差（维度：除通道外）
            dims = (0,) + tuple(range(2, x.dim()))  # N 和空间维度
            batch_mean = x.mean(dim=dims, keepdim=False)
            batch_var = x.var(dim=dims, keepdim=False, unbiased=False)

            # 更新全局统计量（移动平均）
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * batch_mean.detach()
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * batch_var.detach()

            mean, var = batch_mean, batch_var
        else:
            # 推理时使用全局统计量
            mean, var = self.running_mean, self.running_var

        # 标准化：逐通道广播
        x_hat = (x - mean.view(1, -1, *([1] * (x.dim() - 2)))) / torch.sqrt(var.view(1, -1, *([1] * (x.dim() - 2))) + self.eps)
        out = self.gamma.view(1, -1, *([1] * (x.dim() - 2))) * x_hat + self.beta.view(1, -1, *([1] * (x.dim() - 2)))
        return out

BatchNorm、LayerNorm 和 RMSNorm 都是常用的归一化方法，它们的核心区别在于归一化维度与是否去均值。

•	BatchNorm 是在训练时对整个 batch 维度 + 空间维度 做统计，也就是说每个通道共享一组均值和方差，典型用于 CNN。它的优势是能利用 batch 内的统计信息，使训练更稳定、收敛更快。但它依赖 batch size，batch 太小时统计量不稳定，推理时还需要维护全局的 running mean 和 var，这在分布变化或小 batch 场景下会成为劣势。

•	LayerNorm 则是对输入样本的 最后一维（特征维度） 归一化，不依赖 batch 维，因此特别适合 RNN、Transformer 等序列建模任务。优势是对 batch size 不敏感，训练和推理一致；劣势是每个 token 独立归一化，可能弱化了不同样本间的统计作用，对 CNN 效果不如 BatchNorm。

•	RMSNorm 可以看作是 LayerNorm 的一种简化，它只用均方根（RMS）来缩放，不做去均值操作，参数也通常只有缩放因子而没有偏置。这样一方面减少了计算开销，提高数值稳定性，另一方面在残差结构中保留了恒等映射的性质，因此在大模型（如 LLaMA、T5）里很常见。它的不足是缺少去均值操作，在某些任务上表现可能略逊于 LayerNorm。